<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex14notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex14notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 演習課題: 音のデータの解析
---

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>


In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set_theme()

# statsmodels いろいろ
from statsmodels import tsa
from statsmodels.graphics import tsaplots

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 標本化


### 標本化とは

物理世界を観測して得られる情報は，「時間にともなって変化する気温」のように，時間や空間といった数量にともなって変化する何らかの数量であることが多くあります．
何かの数量の変化にともなって変化する何かの数量において，前者（気温の例における時間を表す量）を「変数」といい，後者（気温）を「値」といいます．

ここでの量には，連続なものも離散的な（とびとびの）ものもありますが，コンピュータはディジタルな情報を扱うことしかできませんので，連続な量であっても離散的に表します．
例えば大津市の一地点の気温は，時間にともなって変化する連続な量です．しかし，「データ4: 大津市の2004年の日平均気温」では，「1日ごとの平均気温」という値として，時間軸を1日単位で離散化して扱っていました．

データを扱う際には，このように，本来は連続な変数を離散化してから扱うことがよくあります．変数を離散化するこの処理のことを，**標本化** (sampling) といいます．これに対して，値の方を離散化する処理は **量子化** (quantization) といいます（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 量子化についての説明はこの授業では省略します．データ3 の場合，気温の値は 0.1 度刻みとなっていました．コンピュータで扱う場合は，値を 2 のべき乗通りに離散化する量子化が便利なのでよく用いられます．たとえば一般的な画像ファイルでは画素値を 0 から 255 の 256 段階で表します．
</span>

#### 標本化間隔（標本化周期）と標本化周波数

ここでは，変数を一定の間隔で離散化するような標本化の仕方を考えます．この間隔のことを，**標本化間隔** （または **標本化周期**） といいます．

次のセルをそのまま実行すると，$x(t) = \sin{2\pi t}$ （ $t$ の単位は [s] ）の値を $t = 0, 0.1, 0.2, \ldots$ と標本化間隔 $0.1[s]$ で取り出す標本化を行った結果が表示されます．
灰色の曲線が元の $x(t)$ で，赤い点が標本化によって得られる値です．



In [ ]:
T = 0.1

duration = 2.0

t_sample = np.arange(0, duration, T)
N = len(t_sample)
t = np.arange(0, duration, 0.01)
x_sample = np.sin(2*np.pi*t_sample)
x        = np.sin(2*np.pi*t)

print('x = ')
print(x_sample, f'N = {N}')

fig, ax = plt.subplots(figsize=(8, 4))
ax.stem(t_sample, x_sample, linefmt='r--', markerfmt='ro', label='$x_n$')
ax.plot(t, x, '-', label=f'$\sin(2\pi t)$', color='gray')
ax.set_xlabel('t [s]')
ax.axhline(0, color='gray')
ax.set_xlim(-0.05, 2.05)
ax.set_ylim(-1.2, 1.2)
ax.set_xticks(np.arange(0, duration+0.1, T))
ax.legend()

plt.show()

##### 問題1

上のコードセルでは，標本化間隔を `T` という変数で指定しています．
標本化間隔を $0.12, 0.2, 0.3, 0.4, 0.5$ などと変えて結果を観察しよう．

標本化間隔の逆数をとったものを，**標本化周波数** といいます．標本化間隔の単位が [s] の場合，標本化周波数の単位は [Hz] です．標本化間隔が 0.1[s] なら，標本化周波数は 1/0.1 = 10 [Hz] です．標本化の際には，標本化間隔を短く（標本化周波数を高く）しないと，元の波形をちゃんと表せません（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「では標本化間隔／標本化周波数はどう決めたらよいのだろう」という疑問が浮かびますが，これに関して，「標本化定理」という定理が知られています．例えば，notebookAで出てきた 44100 という数字は，音楽などを記録する CD の規格（CD-DA）で定められた標本化周波数です．標本化定理と人間の聴覚について理解すると，なぜこんな数字なのかが分かるようになります．
</span>

---
### 正弦波で作った音の解析


#### データを準備する

次のコードセルを実行すると，音のデータを作ります．`signalA` と `signalB` の2つあります．それぞれ，ある周波数の正弦波にノイズを乗せた信号2秒分から 1/8000 秒間隔で値を取り出した（標本化間隔 1/8000 秒で標本化した = 標本化周期 8000 Hz で標本化した）ものとなっています．

In [ ]:
sr = 8000        # 標本化周波数
vmax = 2**(15-2) # 最大値を16bit量子化での最大振幅の 1/4 に
T = 2            # 2秒

t = np.linspace(0, T, num=T*sr)
signalA = vmax*(np.sin(800*np.pi*t) + 0.1*np.random.randn(T*sr))
signalB = 0.5*vmax*(np.sin(500*np.pi*t) + 0.1*np.random.randn(T*sr))

##### 問題2

次のセルの 3 行目をコメントにしたがって修正してから実行すると，`signalA` と `signalB` を WAVE 形式のファイルとして出力して手元にダウンロードできます．WAVE 形式のファイルは，Windows でも macOS でも，標準の音楽プレイヤーで再生できるはずです．

どちらの方が高い音でしょうか？どちらの方が大きい音でしょうか？

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    write('signalA.wav', sr, signalA.astype('int16'))
    write('signalB.wav', sr, signalB.astype('int16'))
    files.download('signalA.wav')
    files.download('signalB.wav')

##### 問題3

次のセルを実行すると，`signalA` と `signalB` のグラフを描けます．横軸は，データ点の番号（時系列 $\{x_t\}$ の添字 $t$ ）を表します．2つの音の高さや大きさの違いが波形としてはどのように見えるか考えましょう．

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8))
ax[0].plot(signalA, '-o', label='signalA', markersize=3)
ax[0].axhline(0, color='gray')
ax[0].set_xlabel('t')
ax[0].set_xlim(0, 80)
ax[0].set_ylim(-2**15, 2**15)
ax[0].legend()
ax[1].plot(signalB, '-o', label='signalB', markersize=3)
ax[1].axhline(0, color='gray')
ax[1].set_xlabel('t')
ax[1].set_xlim(0, 80)
ax[1].set_ylim(-2**15, 2**15)
ax[1].legend()
plt.tight_layout()
plt.show()

#### 解析する

##### 問題4

`signalA` と `signalB` の周期/周波数を調べましょう．

次のセルを実行すると，それぞれの自己相関 $\textrm{Corr}[x_t, x_{t-k}]$ を求めてプロットすることができる．横軸はラグ $k$ を表す．それにもとづいて次の問に答えなさい．

(1) `signalA` と `signalB` のそれぞれについて，$k > 0$ で自己相関の値がピークとなる $k$ の最小値はいくつか答えなさい．

(2) これらの信号は，標本化間隔 1/8000 [s] で標本化して得られた時系列である．したがって，ラグの大きさ 1 は 1/8000 [s] に相当する．(1) の結果から推定される `signalA` と `signalB` の周期を [s] の単位で答えなさい．さらに，周波数を [Hz] の単位で答えなさい．





In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
tsaplots.plot_acf(signalA, ax=ax[0], title='Autocorrelation of signalA')
ax[0].set_ylim(-1.1, 1.1)
tsaplots.plot_acf(signalB, ax=ax[1], title='Autocorrelation of signalB')
ax[1].set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.show()

In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'CigxKSBgc2lnbmFsQWAg44Gn44GvIDIwLCBgc2lnbmFsQmAg44Gn44GvIDMyCgooMikgYHNpZ25hbEFgIOOBruWRqOacn+OBryAkXGZyYWN7MjB9ezgwMDB9ID0gXGZyYWN7MX17NDAwfSQgW3Nd77yM5ZGo5rOi5pWw44GvICQ0MDAkIFtIel3vvI4KYHNpZ25hbEJgIOOBruWRqOacn+OBryAkXGZyYWN7MzJ9ezgwMDB9ID0gXGZyYWN7MX17MjUwfSQgW3Nd77yM5ZGo5rOi5pWw44GvICQyNTAkIFtIel3vvI4KCumfs+OCkuiBnuOBhOOBn+e1kOaenOOBqOWQiOOCj+OBm+OCi+OBqO+8jOWRqOazouaVsOOBruWkp+OBjeOBhOato+W8puazouOBruaWueOBjOmrmOOBhOmfs+OBq+iBnuOBk+OBiO+8jOaMr+W5heOBruWkp+OBjeOBhOato+W8puazouOBruaWueOBjOWkp+OBjeOBhOmfs+OBq+iBnuOBk+OBiOOCi+OBk+OBqOOBjOWIhuOBi+OCiuOBvuOBme+8jgo='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

---
### ギター音の解析

ギターの音を録音したデータを解析しよう．


#### 楽器が出す音の性質と音の高さの知覚

ギターのような弦楽器では，両端が固定された弦が振動しそれが空気を振動させることで音が生み出されます（指で弦を押さえることで固定される位置が変わったりもしますね）．このとき，弦の振動は，下図に示すようなパターンを示します．左は両端が固定された弦の振動としては最も波長が長いもの，真ん中は波長がその $\frac{1}{2}$，右は波長 $\frac{1}{3}$ の振動を表しています．これら以外に波長 $\frac{1}{4}, \frac{1}{5}, \ldots$ の振動も現れます．
実際の弦の振動はこれらを合成したものとなりますので，そこから生まれる空気の振動もまた，ある波長の波と，その $\frac{1}{自然数}$ 倍の波長の波が混ざったものとなります．
したがって，弦楽器から生み出される音には，最も波長の長い振動に対応した最も低い周波数（これを基本周波数といいます）の成分と，その自然数倍の周波数の成分（倍音）が含まれます．ちなみに，管楽器の場合は少し様子が異なりますが，その音はやはり基本周波数の成分とその倍音から成ります．

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3))

t = np.linspace(0, np.pi, num=100)

# 基本波長
ax[0].plot(t, np.sin(t), '-', color='red')
ax[0].set_ylim(-1.5, 1.5)
ax[0].plot(t, -np.sin(t), '--', color='red')
ax[0].axhline(0, color='gray')
ax[0].xaxis.set_visible(False)
ax[0].yaxis.set_visible(False)

# 波長 1/2 = 周波数 2 倍
ax[1].plot(t, np.sin(2*t), '-', color='red')
ax[1].set_ylim(-1.5, 1.5)
ax[1].plot(t, -np.sin(2*t), '--', color='red')
ax[1].axhline(0, color='gray')
ax[1].xaxis.set_visible(False)
ax[1].yaxis.set_visible(False)

# 波長 1/3 = 周波数 3 倍
ax[2].plot(t, np.sin(3*t), '-', color='red')
ax[2].set_ylim(-1.5, 1.5)
ax[2].plot(t, -np.sin(3*t), '--', color='red')
ax[2].axhline(0, color='gray')
ax[2].xaxis.set_visible(False)
ax[2].yaxis.set_visible(False)

fig.tight_layout()
plt.show()

一方，人間の聴覚系が知覚する音の高さは基本周波数と強く関連しており，基本的には，基本周波数の高い音ほど高い音に聞こえます．楽器や演奏の仕方が異なると，同じ音の高さでも音色が異なって聞こえますが，これは，基本周波数が同じで，倍音成分の含まれ方が異なるためです．基本周波数の成分がほとんど含まれず周波数2倍以上の倍音成分しか含まれないような音でも，基本周波数に対応した高さの音として知覚されます．

#### データの準備

ネットから WAVE 形式のファイルを入手して使うので，WAVE形式のファイルを読み込む関数を定義しておきます．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):

    framerate, data = read(filename)

    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]

    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'

    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)

    return framerate, dataNew

WAVE ファイルを入手します．

In [ ]:
# WAVE ファイルを入手
#
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar1-C.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar2-E.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar3-G.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar4-ChordC.wav
fnList = ['Sound-Guitar1-C.wav', 'Sound-Guitar2-E.wav', 'Sound-Guitar3-G.wav', 'Sound-Guitar4-ChordC.wav']

import os

for fn in fnList:
    if not os.path.exists(fn):
        print(f'{fn}のダウンロードがうまくできていないようです')

4つのWAVEファイルをダウンロードして聞いてみましょう．
音感の優れたひとは聞いただけで何の音かわかるでしょうが，わからないふりをして先へ進みましょう．
ちなみに，これらの音は，本物のギターを鳴らして録音したものではなく，コンピュータで作り出した音です．ギターの音っぽい倍音成分のパターンを知っていれば，それっぽい波形を合成できる，というわけです．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    for fn in fnList:
        files.download(fn)

4つのWAVEファイルのデータを読み込んで，np.array のリストに格納します．`guitar[0]`, `guitar[1]`, `guitar[2]`, `guitar[3]` のそれぞれが音のデータを格納した1次元配列です．

In [ ]:
guitar = []

for i, fn in enumerate(fnList):
    print(f'##### guitar[{i}] #####')
    fr, dat = readWAVE(fn)
    dat = dat.astype(float)
    dat -= np.mean(dat)
    guitar.append(dat)
    if i == 0:
        N_guitar = len(dat)
        framerate = fr
    else:
        assert len(dat) == N_guitar and fr == framerate
    print()

それぞれの音の信号の波形を眺めてみましょう．

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(16, 10))

t = np.linspace(0, N_guitar/framerate, num=N_guitar, endpoint=False)

for i in range(len(guitar)):
    ax[i, 0].plot(t, guitar[i], label=f'guitar[{i}]')
    ax[i, 0].set_ylim(-150, 150)
    ax[i, 0].legend()
    ax[i, 1].plot(t, guitar[i], 'o', markersize=3, label=f'guitar[{i}]')
    ax[i, 1].axhline(0, color='gray')
    ax[i, 1].set_xlim(0.3, 0.305)
    ax[i, 1].set_ylim(-150, 150)
    ax[i, 1].legend()

fig.tight_layout()
plt.show()

それぞれの図は，横軸を時間（単位は[s]（秒））として音の信号の波形を描いたものです．4つとも2秒強の音です．右の図は，時刻 $0.3$[s] から $0.305[s]$ までの $0.005$ [s] 間のデータ点をプロットしたものです．
これらを見ただけでは，どれがどんな音なのかまるで想像がつきません．


#### 自己相関から周期を推定する

次のコードセルを実行すると，`guitar[0]`, `guitar[1]`, `guitar[2]` に格納された時系列の自己相関を求めてグラフに描きます．

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 8))
for i in range(3):
    tsaplots.plot_acf(guitar[i], ax=ax[i], title=f'Autocorrelation of guitar[{i}]')
    ax[i].set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.show()

#### 問題5


(1) 自己相関におけるラグを $k$ で表すとする．`guitar[0]`, `guitar[1]`, `guitar[2]` のそれぞれについて，$k > 0$ で自己相関の値がピークとなる $k$ の最小値はいくつか答えなさい．

(2) ここで用いているデータは全て標本化周波数 8000 [Hz] （標本化間隔 1/8000 [s]）で標本化されたものである．(1)で求めた値から，それぞれのラグの値が周波数で何[Hz]に相当するか答えなさい．

(3) (2) で求めた周波数がそれぞれの音の基本周波数だとすると，3つの音はそれぞれ何の音か．下記の表を参考に答えなさい（注）．

|基本周波数[Hz]|音名|ハ長調での...|
|---:|:---:|:---:|
|261.6|C4|ド|
|293.7|D4|レ|
|329.6|E4|ミ|
|349.2|F4|ファ|
|392.0|G4|ソ|
|440|A4|ラ|
|493.9|B4|シ|

<span style="font-size: 75%">
※注: ここでは，A4 の基本周波数を 440[Hz]として十二平均律を用いたときの1オクターブの範囲の音の基本周波数の値を示しています．
</span>



ここでは説明しませんが，「フーリエ級数展開」や「フーリエ変換」の離散バージョンである「離散フーリエ変換」を用いると，もっといろいろな分析ができます．音を加工したりすることもできます．